# Adding tracking line to simple plot

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Using moviepy=2.0.0.dev2
from moviepy.editor import VideoClip, AudioFileClip

In [ ]:
video_path = "./example/output/example_simple.mp4"
video_width = 10
video_height = 10
video_fps = 30
line_color = "lime"


# `data` can be any plottable object (e.g., images, mel spectrogram data)
# The data can be also plotted to the axes through alternative means (e.g., librosa.display.specshow)
# Additional plots can be added as needed
data = range(11)

fig, axs = plt.subplots(figsize=(video_width, video_height))

axs.plot(data)

axs.set_xlabel("Time (s)")
axs.set_xlim(0, data.stop - 1)

# # We use blitting to greatly speed up video generation. Save initial draw which will just be the plot(s).
fig.canvas.draw()
bg = fig.canvas.copy_from_bbox(fig.bbox)

width_px, height_px = fig.canvas.get_width_height()  # width, height in pixels


def plot_moving_line(time):
    # Restore background
    fig.canvas.restore_region(bg)

    # Draw line(s)
    # If there are additional plots, a separate line must be drawn for each Axes
    line = axs.axvline(time, color=line_color, animated=True)
    axs.draw_artist(line)

    # blit image and return rgb array (note that we drop the alpha channel)
    fig.canvas.blit(fig.bbox)
    return np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8).reshape(
        height_px, width_px, 4
    )[:, :, :3]


with VideoClip(plot_moving_line, duration=data.stop - 1) as video:
    video.write_videofile(video_path, fps=video_fps)

plt.close(fig)

## Integration with audio_to_mel_spectrogram

In [ ]:
import librosa
import librosa.feature
import librosa.display

In [ ]:
audio_path = "./example/input/example_heart_beat.wav"

video_with_audio_path = "./example/output/example_heart_beat_wav.mp4"

# Generating Spectrogram
n_mels = 128
hop_length = 512
n_fft = hop_length * 2
window = np.hanning(hop_length * 2)

wave, rate = librosa.load(audio_path)
trim, _ = librosa.effects.trim(wave)

mel = librosa.feature.melspectrogram(
    y=trim,
    sr=rate,
    n_mels=n_mels,
    n_fft=n_fft,
    hop_length=hop_length,
    window=window,
)
db = librosa.power_to_db(mel, ref=np.max)

fig, axs = plt.subplots(figsize=(video_width, video_height))

librosa.display.specshow(
    db, x_axis="s", y_axis="mel", sr=rate, ax=axs, n_fft=n_fft, hop_length=hop_length
)


fig.canvas.draw()
bg = fig.canvas.copy_from_bbox(fig.bbox)

width_px, height_px = fig.canvas.get_width_height()


def plot_moving_line(time):
    fig.canvas.restore_region(bg)

    line = axs.axvline(time, color=line_color, animated=True)
    axs.draw_artist(line)

    fig.canvas.blit(fig.bbox)
    return np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8).reshape(
        height_px, width_px, 4
    )[:, :, :3]


with AudioFileClip(audio_path) as audio:
    with VideoClip(plot_moving_line, duration=audio.duration) as video:
        video = video.set_audio(audio)

        video.write_videofile(
            video_with_audio_path,
            fps=video_fps,
            rewrite_audio=False,
            audio=True,
            audio_bitrate="256k",
            audio_codec="aac",
        )

plt.close(fig)